In [40]:
import model
import tf_parser
import importlib
import tensorflow as tf
import random
import numpy as np
import pickle
import char_rnn

In [67]:
importlib.reload(model)
importlib.reload(char_rnn)

<module 'char_rnn' from '/home/gray/scripts/tensorflow-char-rnn/char_rnn.py'>

In [19]:
model.softmax_sample(np.array([0.3, 0.5, 0.3]), temperature = 1)

1

In [20]:
test_dir = '/home/gray/scripts/test/'
model_name = 'grays_model'

In [21]:
test_params = {
    'seq-length' : 50,
    'char-map-size' : 101,
    'label-length' : 50,
    'hidden-layer-sizes' : [100, 100],
    'hidden-layer-details' : [{'activation' : tf.nn.relu}, {'activation' : tf.nn.relu}],
    'minimizer-options' : {'learning-rate' : 0.1}
}

In [22]:
model.create(test_dir, test_params, model_name = model_name)

ValueError: New model directory not empty: /home/gray/scripts/test/

In [ ]:
def character_generator ():
    i = 1 
    for _ in range(0, 1000):
        x = [[ (i + t )% 101 for t in range(0,51)], [(i + t + 45) % 101 for t in range(0, 51)]]
        i += i + random.randint(1, 5)
        
        yield((np.array([s[0:50]for s in x], dtype = np.uint8), np.array([s[1:51]for s in x], dtype = np.uint8)))
         

In [ ]:
for k, _ in character_generator():
    print(k)


In [ ]:
fake_char_map2 = {'a' : 1, 'b' : 2} 
def new_character_generator():
    for _ in range(0, 2000):
        def gen_miniseq():
            seq = [random.choice([1, 2]) for _ in range(0, 4) ]
            for i in range(4, 51):
                if seq[-1] == 1 and seq[-4] == 1:
                    newchar = np.random.choice([1, 2 ], p = [0.9, 0.1])
                if seq[-1] == 1 and seq[-4] == 2:
                    newchar = np.random.choice([1, 2], p = [0.6, 0.4])
                if seq[-1] == 2 and seq[-4] == 2:
                    newchar = np.random.choice([1, 2], p = [0.2, 0.8])
                if seq[-1] == 2 and seq[-4] == 1 :
                    newchar = np.random.choice([1, 2], p = [0.3, 0.7])
                seq.append(newchar)
            return seq
        seqs = [gen_miniseq() for _ in range(0, 26)]
        labels = [s[1:51] for s in seqs]
        chars =  [s[0:50] for s in seqs]
        yield (np.array(chars, dtype = np.uint8), np.array(labels, dtype = np.uint8))

        

In [ ]:
for c, l in new_character_generator():
    print(c[0, :])
    print(l[0, :])

In [ ]:
model.train(test_dir, {'report-freq' : 100, 'state-init' : 'ZERO', 'shapes' : [100, 100]}, character_generator(), model_name )


In [ ]:
fake_char_map = {str(i) : i for i in range(0, 101)}
fake_char_map[' '] = 0
fake_inv_map = {i : str(i) for i in range(0, 101)}
fake_config = {'seq-length' : 50,  'char-map' : fake_char_map, 'inv-char-map' : fake_inv_map, 'state-init' : 'ZERO',
              'temperature' : 1, 'shapes' : [100, 100]}

In [ ]:
model.generate(test_dir, seed_text = '56789', gen_length=200, generate_config = fake_config, model_name=model_name)

In [ ]:
new_test_dir = '/home/gray/scripts/test2/'
new_test_params = {
    'seq-length' : 50,
    'char-map-size' : 3,
    'label-length' : 50,
    'hidden-layer-sizes' : [50, 50],
    'hidden-layer-details' : [{'activation' : tf.nn.relu}, {'activation' : tf.nn.relu}],
    'minimizer-options' : {'learning-rate' : 0.1}
}

In [ ]:
model.create(new_test_dir, new_test_params, model_name = 'abab_model')

In [ ]:
model.train(
    new_test_dir,
    {'report-freq' : 5, 'state-init' : 'ZERO', 'shapes' : [50, 50]},
    new_character_generator(),  'abab_model')

In [28]:
fake_inv_map2 = {1 : 'a', 2 : 'b', 0 : ' '}
fake_char_map2 = {'a' : 1, 'b' : 2, ' ' : 0} 
fake_config2 = {'seq-length' : 50,  'char-map' : fake_char_map2, 'inv-char-map' : fake_inv_map2, 'state-init' : 'ZERO',
              'temperature' : 0.1, 'shapes' : [50, 50]}

In [30]:
model.generate(
    new_test_dir, 
    seed_text = 'bababababababab',
    gen_length=200, 
    generate_config=fake_config2,
    model_name='abab_model')

INFO:tensorflow:Restoring parameters from /home/gray/scripts/test2/abab_model-2000
                                   bababababababab
                                  babababababababb
                                  babababababababb
                                 babababababababbb
                                 babababababababbb
                                babababababababbbb
                                babababababababbbb
                               babababababababbbbb
                               babababababababbbbb
                              babababababababbbbbb
                              babababababababbbbbb
                             babababababababbbbbbb
                             babababababababbbbbbb
                            babababababababbbbbbbb
                            babababababababbbbbbbb
                           babababababababbbbbbbbb
                           babababababababbbbbbbbb
                          babababababababbbbbbbbbb

bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaaaa
bbbbbbbbbbbbbbbbbbbbbbbbbbbaaaa

aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [48]:
with open("/home/gray/scripts/shakespeare/shakespeare.char_map", 'rb') as char_f:
    char_map = pickle.load(char_f)

In [51]:
print([(chr(c), v) for c, v in char_map.items()])

[(' ', 0), ('!', 1), ('$', 2), ('&', 3), ("'", 4), (',', 5), ('-', 6), ('.', 7), ('3', 8), (':', 9), (';', 10), ('?', 11), ('A', 12), ('B', 13), ('C', 14), ('D', 15), ('E', 16), ('F', 17), ('G', 18), ('H', 19), ('I', 20), ('J', 21), ('K', 22), ('L', 23), ('M', 24), ('N', 25), ('O', 26), ('P', 27), ('Q', 28), ('R', 29), ('S', 30), ('T', 31), ('U', 32), ('V', 33), ('W', 34), ('X', 35), ('Y', 36), ('Z', 37), ('a', 38), ('b', 39), ('c', 40), ('d', 41), ('e', 42), ('f', 43), ('g', 44), ('h', 45), ('i', 46), ('j', 47), ('k', 48), ('l', 49), ('m', 50), ('n', 51), ('o', 52), ('p', 53), ('q', 54), ('r', 55), ('s', 56), ('t', 57), ('u', 58), ('v', 59), ('w', 60), ('x', 61), ('y', 62), ('z', 63)]


In [42]:
shk = np.load('/home/gray/scripts/shakespeare2/shakespeare2.npy')

In [46]:
shk[0:100]

array([17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9, 13, 42, 43,
       52, 55, 42,  0, 60, 42,  0, 53, 55, 52, 40, 42, 42, 41,  0, 38, 51,
       62,  0, 43, 58, 55, 57, 45, 42, 55,  5,  0, 45, 42, 38, 55,  0, 50,
       42,  0, 56, 53, 42, 38, 48,  7, 12, 49, 49,  9, 30, 53, 42, 38, 48,
        5,  0, 56, 53, 42, 38, 48,  7, 17, 46, 55, 56, 57,  0, 14, 46, 57,
       46, 63, 42, 51,  9, 36, 52, 58,  0, 38, 55, 42,  0, 38, 49], dtype=uint8)

In [47]:
! cat /home/gray/scripts/shakespeare2/tinyshakespeare.txt | head

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
cat: write error: Broken pipe


In [70]:
it = char_rnn._make_batch_iterator(shk, 50, 32, 2000, 1)

In [72]:
t = 0
for i in it:
    t += 1

IndexError: index 1075394 is out of bounds for axis 1 with size 1075394

In [73]:
t

671